In [1]:
import pandas as pd
import numpy as np

In [2]:
# merge_oil_cpi.csv, price_baechoo_high.csv, transaction_supply_baechoo.csv 파일 불러오기

oil_cpi = pd.read_csv('data/merge_oil_cpi/merge_oil_cpi_daily.csv')
price = pd.read_csv('data/price/price_baechoo.csv')
transaction = pd.read_csv('data/transaction_supply/transaction_supply_baechoo.csv')


In [3]:
oil_cpi.info()
price.info()
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8370 entries, 0 to 8369
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      8370 non-null   object 
 1   자동차용 경유 가격 (원)  8370 non-null   float64
 2   전월비(%)          8370 non-null   float64
dtypes: float64(2), object(1)
memory usage: 196.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5895 entries, 0 to 5894
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      5895 non-null   object 
 1   평균      5895 non-null   float64
dtypes: float64(1), object(1)
memory usage: 92.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7115 entries, 0 to 7114
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   거래년월일   7115 non-null   object 
 1   거래량     7115 non-null   float64
dtypes: float64(1), object(1)
memory usage: 111.3+ KB


In [4]:
# 첫번째 컬럼명 변경 "거래년월일"
oil_cpi.rename(columns={oil_cpi.columns[0] : "거래년월일"}, inplace=True)
oil_cpi.head()

,거래년월일,자동차용 경유 가격 (원),전월비(%)
0,2000-01-01,583.35,4.0
1,2000-01-02,583.35,4.0
2,2000-01-03,583.35,4.0
3,2000-01-04,583.35,4.0
4,2000-01-05,583.35,4.0


In [5]:
# 첫번째 컬럼명 변경 "거래년월일"
price.rename(columns={price.columns[0] : "거래년월일"}, inplace=True)
price.head()

,거래년월일,평균
0,2000-01-03,5600.0
1,2000-01-04,5600.0
2,2000-01-05,5620.0
3,2000-01-06,5700.0
4,2000-01-07,5670.0


In [6]:
transaction.head()

,거래년월일,거래량
0,2000-01-03,102285.0
1,2000-01-04,715338.0
2,2000-01-05,340568.0
3,2000-01-06,290180.0
4,2000-01-07,536736.0


In [7]:
# oil_cpi, price, transaction 데이터프레임 병합
merge = pd.merge(oil_cpi, price, on='거래년월일', how='outer')
merge = pd.merge(merge, transaction, on='거래년월일', how='outer')
# merge 의 2022-12월 데이터 삭제
merge = merge[merge['거래년월일'] < '2022-12-01']
merge


,거래년월일,자동차용 경유 가격 (원),전월비(%),평균,거래량
0,2000-01-01,583.35,4.0,NaN,NaN
1,2000-01-02,583.35,4.0,NaN,NaN
2,2000-01-03,583.35,4.0,5600.0,102285.0
3,2000-01-04,583.35,4.0,5600.0,715338.0
4,2000-01-05,583.35,4.0,5620.0,340568.0
...,...,...,...,...,...
8365,2022-11-26,1879.15,-8.0,NaN,296250.0
8366,2022-11-27,1879.15,-8.0,NaN,NaN
8367,2022-11-28,1879.15,-8.0,5664.0,201478.0
8368,2022-11-29,1879.15,-8.0,5592.0,77130.0


In [8]:
# merge 의 NaN 값의 비율 확인
merge.isnull().sum() / len(merge)


거래년월일             0.000000
자동차용 경유 가격 (원)    0.000000
전월비(%)            0.000000
평균                0.297252
거래량               0.151971
dtype: float64

In [9]:
# NaN 값을 근처값의 평균값으로 대체
merge = merge.fillna(method='ffill')
merge = merge.fillna(method='bfill')
merge

,거래년월일,자동차용 경유 가격 (원),전월비(%),평균,거래량
0,2000-01-01,583.35,4.0,5600.0,102285.0
1,2000-01-02,583.35,4.0,5600.0,102285.0
2,2000-01-03,583.35,4.0,5600.0,102285.0
3,2000-01-04,583.35,4.0,5600.0,715338.0
4,2000-01-05,583.35,4.0,5620.0,340568.0
...,...,...,...,...,...
8365,2022-11-26,1879.15,-8.0,6100.0,296250.0
8366,2022-11-27,1879.15,-8.0,6100.0,296250.0
8367,2022-11-28,1879.15,-8.0,5664.0,201478.0
8368,2022-11-29,1879.15,-8.0,5592.0,77130.0


In [10]:
# 거래년월일 인덱스로 설정
merge.set_index("거래년월일",inplace=True)

In [11]:
# csv 파일로 저장
merge.to_csv('data/merge_all/merge_all.csv', index=True)